# Lesson 3: The LLM as a Translation Engine NOT COMPLETED YET

Welcome to the grand finale! Let's recap what we've built so far:
* **Lesson 1 (RAG):** We taught our LLM how to understand fuzzy human words ("defrosted dunes") using NASA's landform dictionary.
* **Lesson 2 (APIs):** We wrote a Python script to send a strict Lucene query to NASA's server and get back a clickable link to view an image.

Right now, *we* are still the ones doing the heavy lifting. We have to manually figure out the exact Lucene syntax and type it into our Python script. 

In this lesson, we are going to use the LLM to bridge that gap. We will turn the LLM into a **Translation Engine**. Its only job will be to listen to a user's natural language request and translate it into a perfectly formatted Lucene query that our API script can understand.

### Guardrails: The Strict System Prompt
By default, LLMs love to chat. If we ask it for a query, it might say: *"Sure! Here is your query: `...` Let me know if you need anything else!"* Our Python script can't understand that extra text. We need to use a **System Prompt** to strictly forbid the LLM from being conversational.

In [3]:
import os
import requests
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY")
)

# 1. The Strict System Prompt
# We must explicitly tell the LLM not to chat. We ONLY want the raw query!
LUCENE_PROMPT = """
You are a NASA PDS search translator. 
Your job is to translate a user's natural language request into a strict Lucene query.

Use the following fields:
- Mission: gather.common.mission (e.g., "mro", "mars_2020")
- Instrument: gather.common.instrument (e.g., "ctx", "hirise")
- Target: gather.common.target (e.g., "mars", "moon")
- Solar Latitude: pds3_label.SOLAR_LATITUDE (e.g., [0 TO 80])

CRITICAL INSTRUCTION: Return ONLY the Lucene query string. Do not include markdown formatting, quotes, or conversational text.
"""

# 2. The Translation Function
def generate_lucene_query(user_request):
    print(f"User asked: '{user_request}'\n")
    print("🧠 LLM is translating to Lucene...")
    
    response = client.chat.completions.create(
        model="allenai/olmo-3.1-32b-instruct", # A great model for following strict instructions
        messages=[
            {"role": "system", "content": LUCENE_PROMPT},
            {"role": "user", "content": user_request}
        ],
        temperature=0.1 # We keep the temperature very low so it doesn't get "creative"
    )
    
    # .strip() removes any accidental spaces or hidden newlines the LLM might have added
    clean_query = response.choices[0].message.content.strip()
    return clean_query

# Let's test the brain!
user_idea = "Find me MRO Context Camera images of Mars where the solar latitude is between 0 and 50."
generated_query = generate_lucene_query(user_idea)

print(f"\n✅ Generated Query: {generated_query}")

User asked: 'Find me MRO Context Camera images of Mars where the solar latitude is between 0 and 50.'

🧠 LLM is translating to Lucene...

✅ Generated Query: Mission:mro AND Instrument:ctx AND Target:mars AND pds3_label.SOLAR_LATITUDE:[0 TO 50]


### Teaching the LLM about our Tool

Our Python function exists, but the LLM doesn't know it's there. We have to explicitly hand the LLM a "menu" of available tools. 

We do this using a specific JSON format called a **Schema**. The schema tells the LLM the name of the tool, what it does, and what arguments it needs to provide when it wants to use it.

In [4]:
# 2. Define the Tool Schema
# This is the "menu" we hand to the LLM so it knows what it can do.

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_pds_atlas",
            "description": "Searches the NASA Planetary Data System for images based on a Lucene query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "lucene_query": {
                        "type": "string",
                        "description": "The Lucene query string. e.g., 'gather.common.instrument:\"ctx\"'"
                    }
                },
                "required": ["lucene_query"]
            }
        }
    }
]

### The Agent Loop in Action

Now comes the magic. We are going to send a user query to the LLM, but we will attach our `tools` list to the request. 

Pay close attention to the `if response_message.tool_calls:` section. This is the heart of an AI Agent. It's the moment our Python code intercepts the LLM's request, runs the search, and hands the data back so the LLM can finish its thought.

In [6]:
# 3. The Agent Loop
MODEL = "allenai/olmo-3.1-32b-instruct"

def run_pds_agent(user_prompt):
    print(f"User: {user_prompt}\n")
    
    # Step 1: Send the initial prompt AND our tools to the LLM
    messages = [
        {"role": "system", "content": "You are a helpful NASA PDS assistant. Use your tools to find images for the user."},
        {"role": "user", "content": user_prompt}
    ]
    
    print("   [System] 🧠 LLM is thinking...")
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools, # <-- This is where we hand it the menu!
        temperature=0.2
    )
    
    response_message = response.choices[0].message
    
    # Step 2: Did the LLM decide to use a tool?
    if response_message.tool_calls:
        print("   [System] 💡 LLM decided to use a tool!")
        
        # Add the LLM's "tool call" request to the conversation history
        messages.append(response_message)
        
        # Step 3: Execute the tool in Python
        for tool_call in response_message.tool_calls:
            if tool_call.function.name == "search_pds_atlas":
                
                # Extract the lucene query the LLM generated
                function_args = json.loads(tool_call.function.arguments)
                query = function_args.get("lucene_query")
                
                # Actually run our Python function!
                tool_result = search_pds_atlas(query)
                
                # Step 4: Append the result back to the conversation
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": "search_pds_atlas",
                    "content": tool_result
                })
        
        # Step 5: Send the conversation (now containing the tool result) back to the LLM
        print("   [System] 🧠 Sending tool results back to the LLM...")
        final_response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.2
        )
        
        print(f"\nAgent: {final_response.choices[0].message.content}")

    else:
        # If the LLM didn't need a tool, just print its normal text response
        print(f"\nAgent: {response_message.content}")

# Let's try it out!
run_pds_agent("Can you find me a Mars Reconnaissance Orbiter (MRO) image of Mars where the solar latitude is between 0 and 80?")

User: Can you find me a Mars Reconnaissance Orbiter (MRO) image of Mars where the solar latitude is between 0 and 80?

   [System] 🧠 LLM is thinking...


APIStatusError: Error code: 402 - {'error': {'message': 'This request requires more credits, or fewer max_tokens. You requested up to 58836 tokens, but can only afford 54621. To increase, visit https://openrouter.ai/settings/credits and upgrade to a paid account', 'code': 402, 'metadata': {'provider_name': None}}, 'user_id': 'user_38DkHqWZ4gmT325um3LdhNL1hga'}